# Computer Vision

## Assignment

### Imports

In [ ]:
import os
from os import path

import numpy as np

from keras import callbacks
from keras.models import load_model
from segmentation_models import Unet

import src.common as common

### Constants

In [ ]:
BACKBONE = 'vgg16'

data_dir = './data'

default_cp_path = path.join(data_dir, 'cp.ckpt')
default_model_path = path.join(data_dir, 'model.h5')

### Setup

In [ ]:
def setup_args():
    return {
        'cp_path': default_cp_path,
        'input_dir': common.default_input_dir,
        'input_size': common.default_input_size,
        'model_path': default_model_path,
        'update_model': None,
        'cpu': None
    }

### Train

In [ ]:
args = setup_args()

use_cpu = args['cpu']
if use_cpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

cp_path = args['cp_path']
input_dir = args['input_dir']
input_size = args['input_size']
model_path = args['model_path']
update_model = args['update_model']

height, width = input_size

raw_images, raw_masks = common.load_images(input_dir, height, width)

masks = np.expand_dims(raw_masks, axis=3)

(train_x, train_y), (val_x, val_y), (test_x, test_y) =\
    common.split_data(raw_images, masks)

checkpoint_cb = callbacks.ModelCheckpoint(filepath=cp_path,
                                          save_weights_only=True,
                                          verbose=1)

if update_model and model_path:
    model = load_model(model_path)
else:
    model = Unet(BACKBONE,
                 encoder_weights='imagenet',
                 input_shape=(*input_size, 3))
model.compile('Adam', 'binary_crossentropy')

model.fit(train_x, train_y,
          batch_size=2,
          epochs=4,
          validation_data=(val_x, val_y),
          callbacks=[checkpoint_cb])

model.save(model_path)
